In [1]:
import os
import sys
import re
import pandas as pd
import numpy as np
from bisect import bisect_left, bisect_right
import spacy

In [2]:
spacy.require_gpu()

True

In [3]:
nlp = spacy.load("en_core_web_lg")
nlp.disable_pipe("parser")
nlp.enable_pipe("senter")

In [7]:
doc = nlp("""After tokenization, spaCy can parse and tag a given Doc. This is where the trained pipeline and its statistical models come in, which enable 

spaCy to make predictions of which tag or label most likely applies in this 
context. A trained 

component includes binary data that is produced by showing a system enough examples for it to make predictions that generalize across the language - for example, a word following “the” in English is most likely a noun""")

In [10]:
for i, sent in enumerate(doc.sents):
    print(i + 1, sent.text)

1 After tokenization, spaCy can parse and tag a given Doc.
2 This is where the trained pipeline and its statistical models come in, which enable 

spaCy to make predictions of which tag or label most likely applies in this 
context.
3 A trained 

component includes binary data that is produced by showing a system enough examples for it to make predictions that generalize across the language - for example, a word following “the” in English is most likely a noun


In [11]:
script = open("/workspace/mica-text-coref/data/annotation/screenplay/prestige.txt").read()

In [12]:
doc = nlp(script)

In [13]:
len(doc)

39312

In [14]:
for i, sent in enumerate(doc.sents):
    print(i + 1, sent.text)

1 


 
                                THE PRESTIGE





                                 Written by

                     Jonathan Nolan & Christopher Nolan


                         

                         FADE IN:

          TOP HATS.
2 Clustered in a small glade.
3 As we SUPER TITLES,
          a BLACK CAT slinks its way through them.
4 As TITLES END-

           BORDEN (V.O.)
           Are you watching closely?
          A SECOND BLACK CAT races into frame, HISSING, SPITTING,
          CHASING the first cat into the woods beyond and we-

                          CUT TO:

          INT.
5 CLUTTERED WORKSHOP -- DAY

          Moving along a row of CANARIES in CAGES.

           CUTTER (V.O.)
           Every magic trick consists of three
           part, or acts...
          Stop at a cage.
6 Weathered HANDS envelope the canary.
7 Hands
          and voice belongs to a man in his 60's- CUTTER.

           CUTTER (V.O.) (CONT'D)
           The first part is called the Pledge...

In [3]:
movies = [line.split()[0] for line in open("../../data/annotation/movies.txt").read().splitlines()]

for movie in movies:
    print(movie)
    script = open("/workspace/mica-text-coref/data/annotation/screenplay/{}.txt".format(movie)).read().splitlines()
    content = open("/workspace/mica-text-coref/data/annotation/screenplay/{}.txt".format(movie)).read()
    tags = open("/workspace/mica-text-coref/data/annotation/parsed/{}_tags.txt".format(movie)).read().splitlines()

    annotation_df = pd.read_csv("/workspace/mica-text-coref/data/annotation/csv/{}.csv".format(movie), index_col=None)
    clusters = []
    cluster_names = []

    for character, character_df in annotation_df.groupby("entityLabel"):
        cluster_names.append(character)
        clusters.append(sorted([(int(row["begin"]), int(row["end"])) for _, row in character_df.iterrows()]))

    start = np.zeros(len(script), dtype=int)
    is_empty = np.zeros(len(script)).astype(bool)

    for i in range(len(script) - 1):
        start[i + 1] = start[i] + len(script[i]) + 1
        is_empty[i] = len(script[i].strip()) == 0

    is_empty[len(script) - 1] = len(script[len(script) - 1].strip()) == 0
    n = 0
    for i in range(len(script)):
        if i < len(script) - 1:
            line = content[start[i]: start[i + 1]]
        else:
            line = content[start[i]:]
        if script[i].strip() != line.strip():
            n += 1
    n1, n2, n3, n4, n5 = 0, 0, 0, 0, 0

    for cluster in clusters:
        for x, y in cluster:
            i = bisect_right(start, x) - 1
            j = bisect_right(start, y-1) - 1
            if i == j:
                if not is_empty[i]:
                    n1 += 1
                else:
                    n2 += 1
            else:
                mention_tags = tags[i: j + 1]
                if len(set(mention_tags)) == 1:
                    n3 += 1
                else:
                    n4 += 1

            context = "\n".join(script[i: j + 1])
            mention = content[x:y]
            if mention not in context:
                n5 += 1

    print(n1, n2, n3, n4, n5)

    

    print()

avengers_endgame
5000 0 24 1 0

dead_poets_society
3755 0 23 0 0

john_wick
2545 0 35 0 0

prestige
5108 0 31 1 0

quiet_place
2765 0 21 0 0

zootopia
3630 0 47 0 0

